In [1]:
import numpy as np
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from super_learner import*
import statsmodels.api as sm
from scipy.stats import norm, t
from scipy.special import logit, expit
import random
from sklearn.linear_model import ElasticNet, LinearRegression, LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, AdaBoostClassifier, RandomForestClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier

In [19]:
def generate_data(N, outcome_type, treatment_type):

    W1 = np.random.binomial(1, 0.5, N)
    W2 = np.random.binomial(1, 0.65, N)
    W3 = np.round(np.random.uniform(0, 4, N), decimals=3)
    W4 = np.round(np.random.uniform(0, 5, N), decimals=3)

    if treatment_type == 'binary':
        Ap = expit(-0.4 + 0.2*W2 + 0.15*W3 + 0.2*W4 + 0.15*W2*W4)
        A = np.random.binomial(1, Ap, N)
        
    elif treatment_type == 'multigroup':
        Ap1 = expit(-0.4 + 0.2*W2 + 0.3*W3 + 0.1*W4 + 0.4*W2*W4)
        Ap2 = expit(-0.4 + 0.5*W2 + 0.1*W3 + 0.2*W4 + 0.1*W2*W4)
        Ap3 = expit(-0.4 + 0.7*W2 + 0.5*W3 + 0.3*W4 + 0.2*W2*W4)
        Ap4 = expit(-0.4 + 0.1*W2 + 0.2*W3 + 0.4*W4 + 0.1*W2*W4)
        Ap = np.array([Ap1, Ap2, Ap3, Ap4]).T
        Ap = (Ap / Ap.sum(1).reshape(-1,1))
        
        l = [0,1,2,3]
        A = []
        for i in range(len(Ap)):
            ap = Ap[i]
            onehot = np.zeros((4,1))
            choice = random.choices(l, ap)
            onehot[choice] = 1
            A.append(onehot)
        A = np.concatenate(A,1).T


    if outcome_type == 'cls':
        
        if treatment_type  == 'binary':
            Y1p = expit(-1 + 1 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4)
            Y0p = expit(-1 + 0 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4)
            Y1 = np.random.binomial(1, Y1p, N)
            Y0 = np.random.binomial(1, Y0p, N)
            Y = Y1*A + Y0*(1-A)
            
        if treatment_type == 'multigroup':         
            Y3p = expit(-1 + 1 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4)
            Y2p = expit(-1 + 2 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4)
            Y1p = expit(-1 + 0.5 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4)
            Y0p = expit(-1 + 4 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4)
            
            Y3 = np.random.binomial(1, Y3p, N)
            Y2 = np.random.binomial(1, Y2p, N)
            Y1 = np.random.binomial(1, Y1p, N)
            Y0 = np.random.binomial(1, Y0p, N)

            Y = A[:, 3] * Y3 + A[:, 2] * Y2 +  A[:, 1] * Y1 +  A[:, 0] * Y0
            
        
    elif outcome_type == 'reg':
        
        if treatment_type == 2:
            Y1 = -1 + 2*A -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4
            Y0 = -1 + 0 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4

            Y = Y1*A + Y0*(1-A)
        
        if treatment_type == 'multigroup':       
            Y3 = -1 + 1 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4
            Y2 = -1 + 2 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4
            Y1 = -1 + 0.5 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4
            Y0 = -1 + 3 -0.1*W1 + 0.3*W2 + 0.25*W3 + 0.2*W4 + 0.15*W2*W4   

            Y = A[:, 3] * Y3 + A[:, 2] * Y2 +  A[:, 1] * Y1 +  A[:, 0] * Y0
        Y = np.clip(Y, -1, 6)
            
    if treatment_type == 'multigroup':
        data = pd.DataFrame([W1, W2, W3, W4, A, Y, Y0, Y1, Y2, Y3]).T
        data.columns = ['W1', 'W2', 'W3', 'W4', 'Adummy', 'Y', 'Y0','Y1', 'Y2', 'Y3']
        
        A_group = []
        for a in data.Adummy.values:
            val = np.where(a==1)
            A_group.append(val[0])
        A_group = np.concatenate(A_group)
        data['A'] = A_group 
    elif treatment_type == 'binary':
        data = pd.DataFrame([W1, W2, W3, W4, A, Y, Y1, Y0]).T
        data.columns = ['W1', 'W2', 'W3', 'W4', 'A', 'Y', 'Y1','Y0']
    return data
    

In [75]:
outcome_type = 'reg'   # cls or reg
treatment_type = 'multigroup' # binary or multigroup
N = 100
data = generate_data(N=N, outcome_type=outcome_type, treatment_type=treatment_type)

true_psi_1_0 = data.Y1.mean() - data.Y0.mean()
true_psi_2_0 = data.Y2.mean() - data.Y0.mean()
true_psi_3_0 = data.Y3.mean() - data.Y0.mean()
print(true_psi_1_0, true_psi_2_0, true_psi_3_0)


-2.5 -0.9999999999999996 -1.9999999999999998


In [76]:
class TLP(object):
    def __init__(self, data, cause, outcome, confs, precs, QSLdict, GSLdict, outcome_type='reg',
                 outcome_upper_bound=None, outcome_lower_bound=None, multigroup=False):
        
        # general settings
        self.multigroup = multigroup
        self.outcome_type = outcome_type   # reg or cls
        self.outcome_upper_bound = outcome_upper_bound  # for bounded outcomes
        self.outcome_lower_bound = outcome_lower_bound  # for bounded outcomes
        
        # data and variable names
        self.data = data  # pd.DataFrame()
        self.n = len(self.data)
        self.cause = cause
        self.outcome = outcome
        self.confs = confs
        self.precs = precs

        self.Q_X = self.data[list(set(confs)) + list(set(precs)) + [cause]]
        self.G_X = self.data[list(set(confs))]
        self.Q_Y = self.data[outcome].astype('int') if outcome_type == 'cls' else self.data[outcome]
        
        self.A = self.data[cause]
        self.A_dummys = self.A.copy()
        self.A_dummys =  self.A_dummys.astype('category')
        self.A_dummys = pd.get_dummies(self.A_dummys, drop_first=False)
        
        if (self.outcome_upper_bound is not None) and (self.outcome_type == 'reg'):
            self.Q_Y = (self.Q_Y - self.outcome_lower_bound)/(self.outcome_upper_bound - self.outcome_lower_bound)
        
        self.G_Y = self.data[cause]
        self.groups = np.unique(self.A)
        
        
        # Super Learners:
        self.QSLdict = QSLdict
        self.GSLdict = GSLdict
        self.gslr = None
        self.qslr = None
        
        # targeting and prediction storage
        self.Gpreds = None
        self.QAW = None
        self.Qpred_groups = {}
        self.Gpreds = None
        self.first_estimates = {}
        self.first_effects = {}
        self.updated_estimates = {}
        self.updated_effects = {}
        self.clev_covs = {}
        self.epsilons = {}
        self.ses = {}
        self.ps = {}
        

            
    def fit(self, k, standardized_outcome=False):
        
        print('Training G Learners...')
        self.gslr = SuperLearner(output='proba', est_dict=self.GSLdict, k=k, standardized_outcome=False)
        self.gslr.fit(x=self.G_X, y=self.G_Y)
        # PROPENSITY SCORES
        print('Generating G Predictions ')
        self.Gpreds = self.gslr.predict_proba(self.G_X)
        
        print('Training Q Learners...')
        self.qslr = SuperLearner(output=outcome_type, est_dict=self.QSLdict, k=k, standardized_outcome=False)
        self.qslr.fit(x=self.Q_X, y=self.Q_Y)
        
        # QAW SCORES
        print('Generating QAW Predictions ')
        self.QAW = self.qslr.predict(self.Q_X) if self.outcome_type == 'reg' else self.qslr.predict_proba(self.Q_X)[:,1:]
        if self.outcome_upper_bound is not None and self.outcome_type == 'reg':
            print('Bounding outcome predictions.')
            self.QAW = np.clip(self.QAW, 0, 1)
        
        print('SuperLearner Training Completed.')
        
        
    def target_multigroup(self, group_comparisons=None):
        
        # INTERVENTIONAL Y PREDS
        print('Generating Predictions for Counterfactual Outcomes')
        for group in self.groups:
            int_data = self.Q_X.copy()
            int_data[self.cause] = group
            self.Qpred_groups[group] = self.qslr.predict(int_data)
            
        # GROUP DIFFERENCES
        for group_comparison in group_comparisons:
            group_a = group_comparison[0]
            group_ref = group_comparison[1]

            group_a_preds = self.Qpred_groups[group_a]
            group_ref_preds = self.Qpred_groups[group_ref]
            
            difference = (group_a_preds - group_ref_preds)
            self.first_estimates[str(group_comparison)] = difference
            
        # CLEVER COVARIATES
        print('Estimating Clever Covariates')
        for group_comparison in group_comparisons:
            group_a = group_comparison[0]
            group_ref = group_comparison[1]
            group_a_inv_prop = 1 / self.Gpreds[:, group_a]
            group_not_a_inv_prop = - 1 / (1 - self.Gpreds[:, group_a])
            group_clev_cov = ((self.A_dummys.iloc[:, group_a] / self.Gpreds[:, group_a]) - (1 - self.A_dummys.iloc[:, group_a]) / (1 - self.Gpreds[:, group_a])).values 
            self.clev_covs[str(group_comparison)] = (group_clev_cov, group_a_inv_prop, group_not_a_inv_prop)
            
       # ESTIMATE FLUCTUATION PARAMETERS     
        print('Estimating Fluctuation Parameters')
        for group_comparison in group_comparisons:
            group_a = group_comparison[0]
            group_ref = group_comparison[1]
            group_clev_cov = self.clev_covs[str(group_comparison)][0]
            if self.outcome_type == 'cls' or self.outcome_upper_bound is not None:
                eps = sm.GLM(np.asarray(self.Q_Y).astype('float'), group_clev_cov, offset=logit(self.QAW[:,0]), family=sm.families.Binomial()).fit().params[0]
            else:
                eps = (sm.GLM(np.asarray(self.Q_Y).astype('float'), group_clev_cov, offset=self.QAW[:,0]).fit()).params[0]
            self.epsilons[str(group_comparison)] = eps
            
        # UPDATING PREDICTIONS
        print('Updating Initial Counterfactual Predictions')
        for group_comparison in group_comparisons:
            group_a = group_comparison[0]
            group_ref = group_comparison[1]
            group_a_orig = self.Qpred_groups[group_a]
            group_ref_orig = self.Qpred_groups[group_ref]
            
            self.first_effects[str(group_comparison)] = group_a_orig.mean() - group_ref_orig.mean()
            eps = self.epsilons[str(group_comparison)]
            clev_cov_a = self.clev_covs[str(group_comparison)][1]
            clev_cov_ref = self.clev_covs[str(group_comparison)][2]
            
            if self.outcome_type == 'cls' or self.outcome_upper_bound is not None:
                group_a_update = (expit(logit(group_a_orig) + eps * clev_cov_a))
                group_ref_update = (expit(logit(group_ref_orig) + eps * clev_cov_ref))
            else:
                group_a_update = (group_a_orig + eps * clev_cov_a)
                group_ref_update = (group_ref_orig + eps * clev_cov_ref)
            
            self.updated_estimates[str(group_comparison)] = (group_a_update, group_ref_update)
            self.updated_effects[str(group_comparison)] = group_a_update.mean() - group_ref_update.mean()
           
        
        print('Deriving the Influence Function')
        for group_comparison in group_comparisons:
            group_a = group_comparison[0]
            group_ref = group_comparison[1]
            clev_cov_group = self.clev_covs[str(group_comparison)][0]
            ystar_a, ystar_ref = self.updated_estimates[str(group_comparison)][0], self.updated_estimates[str(group_comparison)][1]
            effect_star = self.updated_effects[str(group_comparison)]
            IC =  clev_cov_group.reshape(-1, 1)*(self.Q_Y.values - self.QAW) + (ystar_a - ystar_ref) - effect_star
            IC_var = np.var(IC,ddof=1)
            print(IC.shape)
            se = (IC_var / self.n)**0.5
            p = 2*(1 - t.cdf(np.abs(effect_star)/se, 2*self.n))
            self.ses[str(group_comparison)] = se
            self.ps[str(group_comparison)] = p
            
        
        return self.first_effects, self.updated_effects, self.ses, self.ps
        
        

In [77]:
est_dict_reg = {'Elastic': ElasticNet(), 'SVR': SVR(),
            'LR': LinearRegression(), 'RF': RandomForestRegressor(),
            'MLP':MLPRegressor(alpha=0.001, max_iter=2000), 'AB': AdaBoostRegressor(), 'poly': 'poly'}

est_dict_cls = {'LR':LogisticRegression(max_iter=500), 'MLP':MLPClassifier(alpha=0.001, max_iter=2000), 
            'SVC':SVC(probability=True), 'poly': 'poly', 'RF':RandomForestClassifier(),
            'AB': AdaBoostClassifier()}



In [78]:
k =5
tlp = TLP(data, cause='A', outcome='Y', confs=['W1', 'W2', 'W3', 'W4'],
          precs=[], outcome_type=outcome_type, QSLdict=est_dict_reg, GSLdict=est_dict_cls)

In [79]:
tlp.fit(k=k, standardized_outcome=False)
group_comparisons =[[1,0],[2,0],[3,0]]  # comparison in list format with 'group A [vs] reference_group'
tlp.target_multigroup(group_comparisons=group_comparisons)

Training G Learners...
Training estimator: LR
Training estimator: MLP


/home/matthewvowels/GitHub/PhD_part_1/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/matthewvowels/GitHub/PhD_part_1/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/matthewvowels/GitHub/PhD_part_1/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


Training estimator: SVC
Training estimator: poly
Training estimator: RF
Training estimator: AB
Training estimator on full data: LR
Training estimator on full data: MLP


/home/matthewvowels/GitHub/PhD_part_1/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


Training estimator on full data: SVC
Training estimator on full data: poly
Training estimator on full data: RF
Training estimator on full data: AB
Generating G Predictions 
Training Q Learners...
Training estimator: Elastic
Training estimator: SVR
Training estimator: LR
Training estimator: RF
Training estimator: MLP
Training estimator: AB
Training estimator: poly
Training estimator on full data: Elastic
Training estimator on full data: SVR
Training estimator on full data: LR
Training estimator on full data: RF
Training estimator on full data: MLP
Training estimator on full data: AB
Training estimator on full data: poly
Generating QAW Predictions 
SuperLearner Training Completed.
Generating Predictions for Counterfactual Outcomes
Estimating Clever Covariates
Estimating Fluctuation Parameters
Updating Initial Counterfactual Predictions
Deriving the Influence Function
(100, 100)
(100, 100)
(100, 100)


({'[1, 0]': -1.9686690653793022,
  '[2, 0]': -1.121283946196094,
  '[3, 0]': -1.7860461693414502},
 {'[1, 0]': -2.4291314615965236,
  '[2, 0]': -0.8244598520799435,
  '[3, 0]': -1.8962390437418595},
 {'[1, 0]': 0.1920663376521216,
  '[2, 0]': 0.20448978953648506,
  '[3, 0]': 0.2013644216341761},
 {'[1, 0]': 0.0, '[2, 0]': 7.86813391004948e-05, '[3, 0]': 0.0})

In [80]:
print(true_psi_1_0, true_psi_2_0, true_psi_3_0)

-2.5 -0.9999999999999996 -1.9999999999999998


In [82]:
clev_cov_group = tlp.clev_covs['[1, 0]'][0]
ystar_a, ystar_ref = tlp.updated_estimates['[1, 0]'][0], tlp.updated_estimates['[1, 0]'][1]
effect_star = tlp.updated_effects['[1, 0]']
IC =  clev_cov_group.reshape(-1, 1)*(tlp.Q_Y.values - tlp.QAW) + (ystar_a - ystar_ref) - effect_star
print(IC.shape)

(100, 100)


(100, 4)